In [81]:
import google.datalab.storage as storage
import pandas as pd
from io import BytesIO
from google.datalab import Context
import random, string
import numpy as np
import matplotlib.pyplot as plt

In [82]:
# hide warning message
import warnings
warnings.filterwarnings('ignore')

In [83]:
shared_bucket = storage.Bucket('line_capstone')
for obj in shared_bucket.objects():
  if obj.key.find('/') < 0:
    print(obj.key) # Bucket 내에 포함된 데이터 셋 확인 

clicks_test.csv
clicks_train.csv
documents_categories.csv
documents_entities.csv
documents_meta.csv
documents_topics.csv
events.csv
page_views_sample.csv
promoted_content.csv
sample_submission.csv


In [84]:
my_bucket = storage.Bucket('line_capstone')

docu_meta = my_bucket.object('documents_meta.csv')
uri5 = docu_meta.uri
%gcs read --object $uri5 --variable c_docu_meta
docu_meta = pd.read_csv(BytesIO(c_docu_meta))

pro_cont = my_bucket.object('promoted_content.csv')
uri9 = pro_cont.uri
%gcs read --object $uri9 --variable c_pro_cont
promoted = pd.read_csv(BytesIO(c_pro_cont))

events = my_bucket.object('events.csv')
uri7 = events.uri
%gcs read --object $uri7 --variable c_events
events = pd.read_csv(BytesIO(c_events))

In [85]:
# promoted와 docu_meta를 document_id를 key로 해서 merge
merged = pd.merge(events, docu_meta, how='inner', on='document_id')

In [86]:
# docu_meta에 NaN 값이 이미 많다.
pd.isnull(docu_meta).sum()

document_id           0
source_id          2518
publisher_id      64024
publish_time    1011118
dtype: int64

In [87]:
# evenets의 NaN 값.
pd.isnull(events).sum()

display_id        0
uuid              0
document_id       0
timestamp         0
platform          0
geo_location    340
dtype: int64

In [88]:
# merge했을 때의 NaN 값.
pd.isnull(merged).sum()

display_id            0
uuid                  0
document_id           0
timestamp             0
platform              0
geo_location        340
source_id          1322
publisher_id       1323
publish_time    3566273
dtype: int64

In [89]:
# 필요한 변수들만 할당
new_merged = merged.loc[:,['document_id','timestamp','publish_time']]

In [90]:
# timestamp를 datetime으로 변환시키는 함수 정의
import datetime
def my_date(raw_date):
  added_date = raw_date + 1465876799998 # timestamp에 캐글 공식 설명에 있던 숫자를 더해줌.
  revised_date = pd.to_datetime((added_date // 1000), unit='s') # '밀리초'를 '초'로 바꾸고, string이기 때문에 unit='s' 옵션 설정 후 datetime으로 변환.
  return revised_date

In [91]:
# 위에서 정의한 함수를 사용하여 변환 후 새로운 변수에 할당.
new_merged['new_timestamp'] = my_date(new_merged['timestamp'])

In [92]:
# 변수별 NaN값 개수 확인
new_merged.isnull().sum()

document_id            0
timestamp              0
publish_time     3566273
new_timestamp          0
dtype: int64

In [93]:
# 1970년 1월 1일 이전에 발행된 글은 3,229개이다.
len(new_merged['publish_time'][new_merged['publish_time'] < '1970-01-01 00:00:00'])

3229

In [94]:
# 1970-01-01 이전에 발행된 글들을 모두 1970-01-01로 변경.
new_merged['publish_time'][new_merged['publish_time'] < '1970-01-01 00:00:00'] = '1970-01-01 00:00:00'

In [95]:
# 2016년 6월 28일 이후에 발행된 글들이 23만개나 된다.
len(new_merged['publish_time'][new_merged['publish_time'] > '2016-06-29 00:00:00'])

238741

In [96]:
# 2016년 6월 28일 이후에 발행된 글들을 모두 2016년 6월 28일로 변경.
new_merged['publish_time'][new_merged['publish_time'] > '2016-06-29 00:00:00'] = '2016-06-28 23:59:59'

In [97]:
# object였던 publish_time의 dtype을 datetime64[ns]로 변경.
new_merged['new_publish_time'] = pd.to_datetime(new_merged['publish_time'], format = '%Y-%m-%d %H:%M:%S')

In [103]:
# 변경 후 각 변수별 NaT 값의 개수
new_merged.isnull().sum()

document_id               0
timestamp                 0
publish_time        3566273
new_timestamp             0
new_publish_time    3566273
diff                3566273
dtype: int64

In [99]:
# 이벤트가 발생한 시간에서 글의 발행 날짜를 뺀 값을 diff 열에 할당.
new_merged['diff'] = new_merged['new_timestamp'] - new_merged['new_publish_time']

In [100]:
new_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23120126 entries, 0 to 23120125
Data columns (total 6 columns):
document_id         int64
timestamp           int64
publish_time        object
new_timestamp       datetime64[ns]
new_publish_time    datetime64[ns]
diff                timedelta64[ns]
dtypes: datetime64[ns](2), int64(2), object(1), timedelta64[ns](1)
memory usage: 1.8+ GB


In [ ]:
## 아직 해결하지 못한 것
# publish_time과 timestamp 모두 to_datetime으로 변환할 때, UTC=True를 적용하지 않았음.
# 즉, time range 앞 뒤로 4시간 정도 유의미해 보이는 차이가 있던데, UTC=True를 적용해서 비교해보는 건 어떨까?
# => 시도해보긴 했는데, 변화가 없었던 걸로 기억함.